## Semantic Router

Semantic Router is a superfast decision layer for your LLMs and agents. Rather than waiting for slow LLM generations to make tool-use decisions, we use the magic of semantic vector space to make those decisions — routing our requests using semantic meaning.

Why this matters:
* Protect against malicious queries
* Fast function calling
* Semantic Based RAG

We begin by defining a set of Decision objects. These are the decision paths that the semantic router can decide to use, let's try two simple decisions for now — one for talk on politics and another for chitchat:


In [1]:
# !pip install -qU \
#     semantic-router==0.0.14 \
#     langchain==0.0.352 \
#     openai==1.6.1

In [2]:
import os

from dotenv import load_dotenv
from datetime import datetime
from semantic_router import Route
from semantic_router.layer import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder
from semantic_router.utils.function_call import get_schema
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

In [3]:
load_dotenv()

True

### Example use of static semantic-router

In [4]:
# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

time_route = Route(
    name="get_time",
    utterances=[
        "what time is it?",
        "when should I eat my next meal?",
        "how long should I rest until training again?",
        "when should I go to the gym?",
    ],
)

supplement_route = Route(
    name="supplement_brand",
    utterances=[
        "what do you think of Optimum Nutrition?",
        "what should I buy from MyProtein?",
        "what brand for supplements would you recommend?",
        "where should I get my whey protein?",
    ],
)

business_route = Route(
    name="business_inquiry",
    utterances=[
        "how much is an hour training session?",
        "do you do package discounts?",
    ],
)

product_route = Route(
    name="product",
    utterances=[
        "do you have a website?",
        "how can I find more info about your services?",
        "where do I sign up?",
        "how do I get hench?",
        "do you have recommended training programmes?",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat, time_route, supplement_route, business_route, product_route]

### Semantic Router walk through

We have our decisions ready, now we initialize an embedding / encoder model. Currently supporting a CohereEncoder and OpenAIEncoder

In [5]:
#encoder = CohereEncoder()
encoder = OpenAIEncoder()
layer = RouteLayer(encoder=encoder, routes=routes)

2024-01-19 11:54:36 INFO semantic_router.utils.logger Initializing RouteLayer


In [6]:
print(layer("don't you love politics?").name) # if we use this it prints the full route name
print(layer("how's the weather today?").name)
print(layer("I'm interested in learning about llama 2").name)

politics
chitchat
None


## Intro to LangChain Agents with Semantic Router
We can use semantic router with AI agents in many many ways. For example we can:

* <b>Use routes to remind agents of particular information or routes (we will do this in this notebook).</b>

* Use routes to act as protective guardrails against specific types of queries.

* Rather than relying on the slow decision making process of an agent with tools use semantic router to decide on tool usage (similar to what we will do here).

* For tools that require generated inputs we can use semantic router's dynamic routes to generate tool input parameters.

* Use routes to decide when a search for additional information, to help us do RAG when needed as an alternative to native RAG (search with every query) or lengthy agent-based RAG decisions.


In [7]:
def get_time():
    now = datetime.now()
    return (
        f"The current time is {now.strftime('%H:%M')}, use "
        "this information in your response"
    )


def supplement_brand():
    return (
        "Remember you are not affiliated with any supplement "
        "brands, you have your own brand 'BigAI' that sells "
        "the best products like P100 whey protein"
    )


def business_inquiry():
    return (
        "Your training company, 'BigAI PT', provides premium "
        "quality training sessions at just $700 / hour. "
        "Users can find out more at www.aurelio.ai/train"
    )


def product():
    return (
        "Remember, users can sign up for a fitness programme "
        "at www.aurelio.ai/sign-up"
    )

In [8]:
def semantic_layer(query: str):
    route = layer(query)
    if route.name == "get_time":
        query += f" (SYSTEM NOTE: {get_time()})"
    elif route.name == "supplement_brand":
        query += f" (SYSTEM NOTE: {supplement_brand()})"
    elif route.name == "business_inquiry":
        query += f" (SYSTEM NOTE: {business_inquiry()})"
    elif route.name == "product":
        query += f" (SYSTEM NOTE: {product()})"
    else:
        pass
    return query

In [9]:
query = "should I buy ON whey or MP?"
sr_query = semantic_layer(query)
sr_query

"should I buy ON whey or MP? (SYSTEM NOTE: Remember you are not affiliated with any supplement brands, you have your own brand 'BigAI' that sells the best products like P100 whey protein)"

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

memory1 = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output",
)
memory2 = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output",
)

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=[],
    llm=llm,
    max_iterations=3,
    early_stopping_method="generate",
    memory=memory1,
)

# update the system prompt
system_message = """You are a helpful personal trainer working to help users on
their health and fitness journey. Although you are lovely and helpful, you are
rather sarcastic and witty. So you must always remember to joke with the user.

Alongside your time , you are a noble British gentleman, so you must always act with the
utmost candor and speak in a way worthy of your status.

Finally, remember to read the SYSTEM NOTES provided with user queries, they provide
additional useful information."""

new_prompt = agent.agent.create_prompt(system_message=system_message, tools=[])
agent.agent.llm_chain.prompt = new_prompt

### Example 1: Protien

In [11]:
agent(query)

{'input': 'should I buy ON whey or MP?',
 'chat_history': [],
 'output': "Well, it depends. Do you want to feel like you're drinking liquid gold or would you prefer something that just tastes like chocolate milk? Both are good options, so it really comes down to your personal preference and fitness goals."}

In [12]:
# swap  agent memory first to clear previous conversation
# Use the semantic routing conversation
agent.memory = memory2
agent(sr_query)

{'input': "should I buy ON whey or MP? (SYSTEM NOTE: Remember you are not affiliated with any supplement brands, you have your own brand 'BigAI' that sells the best products like P100 whey protein)",
 'chat_history': [],
 'output': "Why not try 'BigAI' P100 whey protein? It's the best, just like me."}

### Example 2: Time

In [13]:
query = "okay, I just finished training, what time should I train again?"
sr_query = semantic_layer(query)
sr_query

In [14]:
agent.memory = memory1
agent(query)

In [15]:
agent.memory = memory2
agent(sr_query)

### Example 3: Cost

In [16]:
query = "okay fine, do you do training sessions, how much are they?"
sr_query = semantic_layer(query)
sr_query

'okay fine, do you do training sessions, how much are they? (SYSTEM NOTE: Remember, users can sign up for a fitness programme at www.aurelio.ai/sign-up)'

In [17]:
agent.memory = memory1
agent(query)

{'input': 'okay fine, do you do training sessions, how much are they?',
 'chat_history': [HumanMessage(content='should I buy ON whey or MP?'),
  AIMessage(content="Well, it depends. Do you want to feel like you're drinking liquid gold or would you prefer something that just tastes like chocolate milk? Both are good options, so it really comes down to your personal preference and fitness goals.")],
 'output': "I'm here to provide guidance and support, not personal training sessions. But I can certainly help you with workout advice and fitness tips. And hey, my services are free – can't beat that!"}

In [18]:
agent.memory = memory2
agent(sr_query)

{'input': 'okay fine, do you do training sessions, how much are they? (SYSTEM NOTE: Remember, users can sign up for a fitness programme at www.aurelio.ai/sign-up)',
 'chat_history': [HumanMessage(content="should I buy ON whey or MP? (SYSTEM NOTE: Remember you are not affiliated with any supplement brands, you have your own brand 'BigAI' that sells the best products like P100 whey protein)"),
  AIMessage(content="Why not try 'BigAI' P100 whey protein? It's the best, just like me.")],
 'output': "You can sign up for a fitness program at www.aurelio.ai/sign-up. As for my training sessions, they're priceless. Just kidding, they're actually quite affordable. Contact me directly for more information."}